Imports

In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

**Default Values**

In [2]:
path = "/Users/vanessalieberg/code/Megeve2022/Finance_python/data_stocks/"

# Start date defaults
S_YEAR = 2019
S_MONTH = 1
S_DAY = 3
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2022
E_MONTH = 4
E_DAY = 1
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

**Get Stock File Names in a List**

In [3]:
# listdir returns all files in the directory and isfile will return true
# if it is a file and then we store its name in our list named files
files = [x for x in listdir(path) if isfile(join(path, x))]

# Remove extension from file names
# Splitext splits the file name into 2 parts being the name and extension
# We say get all file names and then store just the name in our list named files
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['CSCO',
 'PRI',
 'HLNE',
 'HUBS',
 'SBGI',
 'UFCS',
 'GFED',
 'AIRT',
 'SITC',
 'RHE',
 'ISSC',
 'TRNS',
 'AEIS',
 'UAL',
 'TROW',
 'ISRG',
 'EGP',
 'EFC',
 'EEX',
 'NVR',
 'PLYM',
 'FEIM',
 'WOR',
 'PRGO',
 'LECO',
 'RPT',
 'VCYT',
 'COHN',
 'PLOW',
 'UVE',
 'ORRF',
 'OESX',
 'SIBN',
 'NOG',
 'BXC',
 'CHCO',
 'LBTYK',
 'SPOK',
 'TPR',
 'DVN',
 'MGEE',
 'AMSC',
 'SVT',
 'SCYX',
 'MN',
 'LOVE',
 'ASYS',
 'INSE',
 'HAFC',
 'CR',
 'ADTN',
 'JVA',
 'JNCE',
 'OFLX',
 'DRRX',
 'BV',
 'AM',
 'W',
 'CE',
 'ASTC',
 'MRO',
 'HOV',
 'NBIX',
 'EVTC',
 'VYGR',
 'BA',
 'VRTX',
 'AFG',
 'GILD',
 'WW',
 'MDB',
 'MGY',
 'EVBN',
 'SLM',
 'NLSN',
 'AMKR',
 'VPG',
 'THC',
 'FCFS',
 'VRAY',
 'LOOP',
 'TAYD',
 'MDU',
 'XELA',
 'TDW',
 'WSTG',
 'CEI',
 'CFR',
 'PFMT',
 'DCO',
 'TVTY',
 'EQIX',
 'ZG',
 'HVT',
 'MKL',
 'QUIK',
 'HUN',
 'MBOT',
 'TER',
 'CULP',
 'SMBK',
 'JOB',
 'GPMT',
 'PODD',
 'MDT',
 'ARC',
 'ATGE',
 'IPAR',
 'DLA',
 'STAR',
 'SND',
 'AXSM',
 'BOOT',
 'MDC',
 'AMSWA',
 'FORM',
 'CPIX',
 'V

**Create a DataFrame from our List**

In [4]:
stock_df = pd.DataFrame(tickers,columns=['Ticker'])
stock_df

,Ticker
0,CSCO
1,PRI
2,HLNE
3,HUBS
4,SBGI
...,...
2890,PPL
2891,OBCI
2892,ROKU
2893,MTDR


**Function that Return a DataFrame from a CSV**

In [5]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(path + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

**Function that Saves Dataframe to CSV**

In [6]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(path + ticker + '.csv')

**Delete Unamed Columns in CSV Files**

In [7]:
def delete_unnamed_cols(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

**Add Daily Return to Dataframe**

In [8]:
# We calculate a percentage rate of return for each day to compare investments.
# Simple Rate of Return = (End Price - Beginning Price) / Beginning Price OR (EP / BP) - 1

# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    df.to_csv(path + ticker + '.csv')
    return df  

**Returns Return on Investment over Time**

Return on Investment is the return you receive from your investment
This amount does not include your initial investment

If you invest 100 and have 200 after 5 years
ROI = end value (200) - Initial value (100) / Initial value  = 1
your new total is Initial Investment + 1* Initial Investment = 200

In [9]:
# Return on Investment is the return you received from your investment
# This amount does not include your initial investment
# If you invest 100 and have 200 after 5 years
# ROI = End Value (200) - Initial Value (100) / Inital Value = 1
# Your new total is Inital Investment + 1 * Inital Investment = 200

def get_roi_defined_time(df):
    # Set as a datetime
    df['Date'] = pd.to_datetime(df['Date'])
    start_val = df[df['Date'] == S_DATE_STR]['Adj Close'][0]
    print("Initial Price :", start_val)
    
    # ----- I CHANGED THIS AFTER THE VIDEO -----
    
    end_val = df[df['Date'] == E_DATE_STR]['Adj Close']
    print(end_val.item())
    print("Final Price :", end_val.item())
    
    # ----- END OF VIDEO CHANGES -----
    
    # Calculate return on investment
    roi = (end_val - start_val) / start_val

    # Return the total return between 2 dates
    return roi

**Get Coefficient of Variation**

In [10]:
# Receives the dataframe with the Adj Close data and returns the coefficient of variation
def get_cov(stock_df):
    mean = stock_df['Adj Close'].mean()
    sd = stock_df['Adj Close'].std()
    cov = sd / mean
    return cov

**Test Functions**

In [12]:
# Create a backup for all original stock data

# Get our 1st ticker
tickers[0]

# Get a dataframe for that ticker
stock_a = get_df_from_csv(tickers[0])
stock_a

# Add daily return to this dataframe
add_daily_return_to_df(stock_a, tickers[0])
stock_a

# Delete unnamed columns in dataframe
stock_a = delete_unnamed_cols(stock_a)
stock_a


# Save cleaned dataframe to csv
save_dataframe_to_csv(stock_a, tickers[0])

**Add Daily Returns & Clean Up All Files**

In [13]:
# Create a backup for all original stock data

# Cycle through all tickers
for ticker in tickers:
    print("Working on :", ticker)
    
    # Get a dataframe for that ticker
    stock_df = get_df_from_csv(ticker)
    
    # Add daily return to this dataframe
    add_daily_return_to_df(stock_df, ticker)
    
    # Delete unnamed columns in dataframe
    stock_df = delete_unnamed_cols(stock_df)
    
    # Save cleaned dataframe to csv
    save_dataframe_to_csv(stock_df, ticker)

Working on : CSCO
Working on : PRI
Working on : HLNE
Working on : HUBS
Working on : SBGI
Working on : UFCS
Working on : GFED
Working on : AIRT
Working on : SITC
Working on : RHE
Working on : ISSC
Working on : TRNS
Working on : AEIS
Working on : UAL
Working on : TROW
Working on : ISRG
Working on : EGP
Working on : EFC
Working on : EEX
Working on : NVR
Working on : PLYM
Working on : FEIM
Working on : WOR
Working on : PRGO
Working on : LECO
Working on : RPT
Working on : VCYT
Working on : COHN
Working on : PLOW
Working on : UVE
Working on : ORRF
Working on : OESX
Working on : SIBN
Working on : NOG
Working on : BXC
Working on : CHCO
Working on : LBTYK
Working on : SPOK
Working on : TPR
Working on : DVN
Working on : MGEE
Working on : AMSC
Working on : SVT
Working on : SCYX
Working on : MN
Working on : LOVE
Working on : ASYS
Working on : INSE
Working on : HAFC
Working on : CR
Working on : ADTN
Working on : JVA
Working on : JNCE
Working on : OFLX
Working on : DRRX
Working on : BV
Working on : 

Working on : OSUR
Working on : HCSG
Working on : ACRX
Working on : PDLB
Working on : AXGN
Working on : HMN
Working on : MPW
Working on : TSN
Working on : TXMD
Working on : GNRC
Working on : QRTEA
Working on : WTFC
Working on : FLR
Working on : RBCN
Working on : NR
Working on : TPB
Working on : AMPH
Working on : NE
Working on : ODP
Working on : UBCP
Working on : PEP
Working on : TGLS
Working on : NSIT
Working on : AERI
Working on : MOFG
Working on : FNCB
Working on : PEG
Working on : GFF
Working on : RARE
Working on : DAKT
Working on : SNDX
Working on : ABTX
Working on : ANAT
Working on : SRTS
Working on : NOW
Working on : COKE
Working on : IDA
Working on : GHM
Working on : RPD
Working on : EYPT
Working on : ESE
Working on : CADE
Working on : GCBC
Working on : PLNT
Working on : APPN
Working on : LLY
Working on : LOB
Working on : LEVI
Working on : FBMS
Working on : MASI
Working on : TPTX
Working on : ONVO
Working on : EGAN
Working on : UMH
Working on : ALNY
Working on : FISI
Working on :

Working on : CLF
Working on : FRD
Working on : ZSAN
Working on : NKSH
Working on : TNC
Working on : TCS
Working on : CCI
Working on : CLDX
Working on : JHG
Working on : CEMI
Working on : FMAO
Working on : MMM
Working on : SEB
Working on : ATLC
Working on : CRMD
Working on : NNBR
Working on : CBZ
Working on : MLI
Working on : QUMU
Working on : DWSN
Working on : KVHI
Working on : MOS
Working on : FTNT
Working on : NUVA
Working on : MOD
Working on : BATRK
Working on : AMCX
Working on : GERN
Working on : HSY
Working on : JNPR
Working on : GIII
Working on : CMC
Working on : CNX
Working on : AXTI
Working on : AVYA
Working on : DHI
Working on : CNO
Working on : CMT
Working on : RM
Working on : MTN
Working on : ED
Working on : HIW
Working on : FFIN
Working on : STRS
Working on : HHS
Working on : CBSH
Working on : ES
Working on : SAIA
Working on : JBSS
Working on : ADSK
Working on : QLYS
Working on : FOCS
Working on : SFNC
Working on : CBRL
Working on : RELV
Working on : ONB
Working on : CUE
Wo

Working on : SYNA
Working on : BRG
Working on : SWAV
Working on : KZR
Working on : UAMY
Working on : SEIC
Working on : TRIP
Working on : PETS
Working on : KOD
Working on : RAMP
Working on : BEN
Working on : KOS
Working on : DISCK
Working on : TRHC
Working on : OPTT
Working on : UEC
Working on : ICPT
Working on : UHS
Working on : LEA
Working on : SBCF
Working on : RLJ
Working on : PTN
Working on : TBBK
Working on : NFBK
Working on : CCBG
Working on : BKE
Working on : FIVE
Working on : TISI
Working on : ROLL
Working on : VXRT
Working on : FLIC
Working on : BKH
Working on : EMN
Working on : GTN
Working on : SBAC
Working on : ROK
Working on : SIRI
Working on : PTC
Working on : GTY
Working on : WRE
Working on : ARLO
Working on : FWRD
Working on : NRG
Working on : PRTS
Working on : ARMK
Working on : NSC
Working on : AROC
Working on : MATW
Working on : MTOR
Working on : NXGN
Working on : WNEB
Working on : PNFP
Working on : GOGO
Working on : UCTT
Working on : OPTN
Working on : NPO
Working on :

Working on : CLIR
Working on : MCK
Working on : MGNX
Working on : AVNS
Working on : TXT
Working on : AAN
Working on : MTB
Working on : FDS
Working on : AAME
Working on : RLGT
Working on : VAC
Working on : HIL
Working on : AXAS
Working on : WATT
Working on : MRTN
Working on : WTBA
Working on : MYE
Working on : VMW
Working on : CVR
Working on : LSBK
Working on : POWI
Working on : CLRO
Working on : EXPI
Working on : SSB
Working on : SNBR
Working on : ANDE
Working on : PBI
Working on : PAR
Working on : TRST
Working on : EBMT
Working on : ANGI
Working on : TLYS
Working on : IART
Working on : CVNA
Working on : UMBF
Working on : CABO
Working on : KREF
Working on : NHTC
Working on : APAM
Working on : RDFN
Working on : EVC
Working on : ALSN
Working on : COOP
Working on : UCBI
Working on : SENEB
Working on : AKBA
Working on : ABIO
Working on : BERY
Working on : FRME
Working on : DXYN
Working on : DISCA
Working on : ALEC
Working on : CTMX
Working on : AKAM
Working on : EBF
Working on : VSAT
Worki

Working on : CHD
Working on : MNTX
Working on : SLG
Working on : HTA
Working on : NEWR
Working on : BANC
Working on : ZM
Working on : TEN
Working on : ESPR
Working on : INFN
Working on : AMED
Working on : STOR
Working on : PCRX
Working on : XENT
Working on : PAHC
Working on : CFX
Working on : MHK
Working on : CLBS
Working on : SFST
Working on : TEX
Working on : CPRX
Working on : BSRR
Working on : PHAS
Working on : WSTL
Working on : LOCO
Working on : ADMP
Working on : OPY
Working on : HSON
Working on : SCI
Working on : DAL
Working on : RESI
Working on : HASI
Working on : CIA
Working on : CHE
Working on : TG
Working on : APA
Working on : ARI
Working on : VRS
Working on : OCFC
Working on : MFA
Working on : WK
Working on : FCF
Working on : SXC
Working on : FFNW
Working on : XHR
Working on : IBKR
Working on : UBOH
Working on : K
Working on : IROQ
Working on : JWN
Working on : SXT
Working on : MESA
Working on : AFL
Working on : VRTS
Working on : BJ
Working on : ADVM
Working on : VGR
Working 

**Get Stock Return over Time Period & Coefficient of Variation**

In [15]:
stock_a

# Get total return since 2017
# Final Price 167.67 = (44.77 * 2.745) + 44.77
get_roi_defined_time(stock_a)

# Get coefficient of variation 
# This is higher than normal because I'm using many years instead of one
get_cov(stock_a)

KeyError: 0